## 라이브러리 호출

In [1]:
# Pandas 라이브러리 임포트
import pandas as pd
from dotenv import load_dotenv
import os
import xml.etree.ElementTree as ET 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

load_dotenv()

True

## API 요청

In [2]:
import requests

SEOUL_DATA_API_KEY = os.getenv('SEOUL_DATA_API_KEY')

url = f'http://openapi.seoul.go.kr:8088/{SEOUL_DATA_API_KEY}/xml/tbLnOpendataRtmsV/1/1000'

In [3]:
try:
    response = requests.get(url)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"API 요청 중 오류 발생: {e}")

## XML -> DataFrame 형태로 변환

In [4]:
try:
    # 응답 내용을 XML로 파싱
    root = ET.fromstring(response.content)

    # 데이터를 저장할 리스트 초기화
    data = []

    # 제공된 API 문서의 출력 필드명과 원하는 한글 컬럼명 매핑
    # {API 필드명: 한글 컬럼명} 형태의 딕셔너리 사용
    field_mapping = {
        'RCPT_YR': '접수연도',
        'CGG_CD': '자치구코드',
        'CGG_NM': '자치구명',
        'STDG_CD': '법정동코드',
        'STDG_NM': '법정동명',
        'LOTNO_SE': '지번구분',
        'LOTNO_SE_NM': '지번구분명',
        'MNO': '본번',
        'SNO': '부번',
        'BLDG_NM': '건물명',
        'CTRT_DAY': '계약일',
        'THING_AMT': '물건금액(만원)',
        'ARCH_AREA': '건물면적(㎡)',
        'LAND_AREA': '토지면적(㎡)',
        'FLR': '층',
        'RGHT_SE': '권리구분',
        'RTRCN_DAY': '취소일',
        'ARCH_YR': '건축년도',
        'BLDG_USG': '건물용도',
        'DCLR_SE': '신고구분',
        'OPBIZ_RESTAGNT_SGG_NM': '신고한 개업공인중개사 시군구명'
    }

    # 각 'row' 태그를 순회하며 데이터 추출
    for row in root.findall('.//row'): # './/row'는 root 아래의 모든 'row' 태그를 찾습니다.
        # print(ET.tostring(row, encoding='utf-8').decode('utf-8')) # 각 row의 XML 내용을 출력하려면 이 주석을 해제하세요.
        entry = {}
        # 매핑된 필드 정보를 사용하여 데이터 추출
        for api_field, korean_name in field_mapping.items():
            # 해당 API 필드 태그를 찾아서 텍스트 추출. 태그가 없을 경우 None으로 처리.
            element = row.find(api_field)
            entry[korean_name] = element.text if element is not None else None # 한글 컬럼명을 키로 사용

        data.append(entry) # 각 row에서 추출한 데이터를 data 리스트에 추가

    # 추출한 데이터를 Pandas DataFrame으로 변환
    df_api = pd.DataFrame(data)

    # DataFrame의 처음 5행 출력하여 확인
    print("\nAPI 응답을 DataFrame으로 변환 성공")

except ET.ParseError as e:
    print(f"XML 파싱 중 오류 발생: {e}")
except Exception as e:
    print(f"처리 중 예상치 못한 오류 발생: {e}")


API 응답을 DataFrame으로 변환 성공


In [5]:
df_api

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,건물명,...,물건금액(만원),건물면적(㎡),토지면적(㎡),층,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명
0,2025,11650,서초구,10800,서초동,1,대지,1327,0027,강남역 한화오벨리스크,...,26300,33.575,45.283000,10,None,None,2003,오피스텔,중개거래,서울 서초구
1,2025,11260,중랑구,10100,면목동,1,대지,0375,0003,면목골드빌라,...,24800,57.2,28.000000,5,None,None,2002,연립다세대,중개거래,서울 중랑구
2,2025,11650,서초구,10800,서초동,1,대지,1327,0027,강남역 한화오벨리스크,...,26300,33.575,45.283000,10,None,20250423,2003,오피스텔,중개거래,서울 서초구
3,2025,11350,노원구,10500,상계동,1,대지,1410,0000,노원롯데캐슬시그니처,...,87000,59.975,0.000000,15,None,None,2023,아파트,중개거래,서울 노원구
4,2025,11740,강동구,10900,천호동,1,대지,0332,0019,광나루에스엠파크,...,26000,29.09,15.000000,6,None,None,2018,연립다세대,직거래,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2025,11590,동작구,10700,사당동,None,None,None,None,None,...,223155,248.46,159.000000,None,None,None,1995,단독다가구,직거래,None
996,2025,11545,금천구,10300,시흥동,1,대지,1013,0000,관악산벽산타운5,...,42800,59.34,0.000000,6,None,None,2004,아파트,중개거래,서울 금천구
997,2025,11230,동대문구,10500,답십리동,1,대지,0010,0000,동아,...,85700,114.84,0.000000,5,None,None,2001,아파트,중개거래,서울 동대문구
998,2025,11380,은평구,10700,응암동,1,대지,0742,0006,청송하이츠빌라,...,25500,48.72,34.000000,2,None,None,1989,연립다세대,중개거래,서울 은평구


## 간단한 시각화

### 1. 법정동명별 거래 건수 시각화 (상위 15개)

In [6]:
fig1 = px.bar(df_api['법정동명'].value_counts().reset_index().head(15), 
             x='법정동명', y='count', 
             title='법정동명별 거래 건수 (상위 15개)',
             color='count')
fig1.update_layout(xaxis_title='법정동명', yaxis_title='거래 건수')
fig1.show()

### 2. 건물용도별 거래 건수 시각화

In [8]:
fig2 = px.pie(df_api, names='건물용도', title='건물용도별 거래 비율')
fig2.update_traces(textposition='inside', textinfo='percent+label')
fig2.show()


# 프로젝트 제안

## 주제: AI 기반 데이터 연계 및 공간 분석을 통한 서울시 부동산 실거래가 정보의 활용성 증대

## 프로젝트 개요:
서울시 부동산 실거래가 정보는 중요한 데이터이지만, 위치 좌표(위도, 경도)가 포함되어 있지 않아 공간 분석에 제약이 있습니다. 또한, 주소 정보의 비표준화로 인해 다른 공간 데이터와의 연계가 어렵다는 한계가 있습니다. 본 프로젝트에서는 AI 기술을 활용하여 실거래가 데이터에 위치 정보를 추가하고, 이를 통해 데이터의 활용성을 높이는 방법을 탐구합니다.

## 데이터 수집:

- 무엇을 위해 어떤 데이터가 필요한가?

  - 목적: 서울시 부동산 실거래가 데이터의 공간적 패턴 분석, 지역별 특성 연계 분석 (예: 지하철역 접근성, 학군 등), 시각화 강화를 위해 위치 좌표(위도, 경도) 데이터가 필요합니다.

  - 필요 데이터:

    - 서울시 부동산 실거래가 정보 (Open API 활용하여 전체 데이터 수집)

    - 서울시 건물 주소 및 위치 좌표 데이터 (다른 공공 데이터 포털 등에서 확보 가능한 데이터셋 탐색)

    - 분석 목적에 따라 추가적인 공간 데이터 (예: 지하철역 위치, 행정구역 경계 등)

- 기존의 데이터의 한계는 무엇인가?

    - 위치 정보 부재: 실거래가 데이터 자체에 위도, 경도 정보가 없어 공간 분석이 불가능합니다.

- 어떻게 유용한 데이터를 구축할 것인가? (Using Generative AI)

    - AI 기반 주소 파싱 및 표준화 모델 개발/활용: 실거래가 데이터의 주소 문자열을 구조화된 형태로 파싱하고 표준화합니다.

    - AI 기반 주소 매칭 알고리즘 적용: 표준화된 주소 정보를 활용하여 건물 주소/위치 좌표 데이터셋과 실거래가 데이터를 매칭합니다. 불완전하거나 약간 다른 주소도 매칭할 수 있도록 퍼지 매칭(Fuzzy Matching) 또는 임베딩 기반 유사도 측정 등 AI/ML 기법을 활용합니다.

    - 매칭된 결과를 기존 실거래가 데이터에 추가하여 '위도', '경도' 컬럼을 포함하는 새로운 통합 데이터셋을 구축합니다.

- 어떻게 유용한 데이터인지를 증명할 것인가?

  - 시각화: 구축된 통합 데이터셋을 활용하여 서울시 지도 위에 실거래가 정보를 시각화합니다 (예: 거래량/가격 히트맵, 특정 건물 유형 분포). 기존 데이터로는 불가능했던 공간적 시각화를 통해 데이터의 유용성을 보여줍니다.

  - 공간 분석: 위치 정보를 활용하여 지역별 평균 가격, 특정 시설(지하철역 등)과의 거리에 따른 가격 변화 등 공간 분석 결과를 제시합니다.

  - 데이터 품질 개선 효과: AI 매칭을 통해 얼마나 많은 실거래가 데이터에 정확한 위치 정보가 추가되었는지 정량적으로 보여줍니다.

- 데이터가 고르게 잘 분포되어 있는가?

  - 수집된 전체 데이터의 시간적(연도별, 월별) 및 공간적(자치구별, 법정동별) 분포를 분석합니다.

- 합법적 데이터인가?

  - 서울시 열린데이터광장에서 제공하는 데이터는 '공공누리 1유형(출처표시, 상업적 이용 및 변경 가능)' 라이선스를 따르므로 합법적으로 사용 및 변경 가능합니다.

  - 다른 공공 데이터 포털에서 확보하는 데이터 또한 라이선스를 확인하여 합법적으로 사용합니다.
